# DC Medium Office Metamodel

In [12]:
# import libraries

import os
import pandas as pd
import plotly.express as px
import json

calibration_results = os.path.join("output", "calibration_results")
if not os.path.exists(calibration_results):
    os.makedirs(calibration_results)

OPTIMIZE_VERSION = 'v4'
BUILDINGS = [
    {
        "building_id": 0,
        "building_address": "1146 19th Street",
        
    },
    {
        "building_id": 1,
        "building_address": "3307 - 3311 M ST NW",
    },
    {
        "building_id": 2,
        "building_address": "1235 W Street NE"
    },
]


In [13]:
for building in BUILDINGS:
    # read in the JSON history data
    history_path = os.path.join(
        'output', f'optimize_{OPTIMIZE_VERSION}', f'building_{building["building_id"]}', 'history.json' 
    )
    data = json.load(open(history_path, 'r'))

    # Convert the historic data into a dataframe
    # "lpd": 15.830236975337932,
    #       "number_of_stories": 2
    #     },
    #     "outputs": {
    #       "electricity_hat": 521594.74259882374,
    #       "gas_hat": 2964.535457082686,
    #       "eui_hat": 44.09521074390874,
    #       "electricity_error": 292496.1574011763,
    #       "gas_error": 2964.535457082686,
    #       "eui_error": 14.904789256091263
    new_data = {
        'index': [], 
        'lpd': [],
        'number_of_stories': [],
        'aspect_ratio': [],
        'hours_of_operation_start': [],
        'hours_of_operation_duration': [],
        'electricity_error': [],
        'gas_error': [],
        'eui_error': [],
        'hover': []
    }
    
    for j in data:
        new_data['index'].append(j['index'])
        new_data['lpd'].append(j['inputs']['lpd'])
        new_data['number_of_stories'].append(j['inputs']['number_of_stories'])
        new_data['aspect_ratio'].append(j['inputs']['aspect_ratio'])
        new_data['hours_of_operation_start'].append(j['inputs']['hours_of_operation_start'])
        new_data['hours_of_operation_duration'].append(j['inputs']['hours_of_operation_duration'])
        new_data['electricity_error'].append(j['outputs']['electricity_error'])
        new_data['gas_error'].append(j['outputs']['gas_error'])
        new_data['eui_error'].append(j['outputs']['eui_error'])
        new_data['hover'].append(f"Index {j['index']}, \n"
                                 f"LPD {j['inputs']['lpd']}, \n"
                                 f"NoS {j['inputs']['number_of_stories']}, \n"
                                 f"AR {j['inputs']['aspect_ratio']}, \n"
                                 f"HoO Start {j['inputs']['hours_of_operation_start']}, \n"
                                 f"HoO Dur {j['inputs']['hours_of_operation_duration']}"
                                )    
    df = pd.DataFrame(new_data)

    # create the inline figure so that we can select which ID to use
    fig = px.scatter(
        df, x="electricity_error", y="gas_error", size_max=60, hover_name='hover'
    )
    
    fig.update_layout(
        height=800,
        title_text=f'Building {building["building_id"]} Pareto Front, {building["building_address"]}',
        xaxis_title_text=f"Error in Electricity Calibration",
        yaxis_title_text='Error in Gas Calibration',
    )
    
    fig.show()
    fig.write_image(f"{calibration_results}/Building {building['building_id']}.png")